In [1]:
# py37NLP
# Check GPU
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices()) 

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 994767211441171191
]


In [2]:
# conda install -c conda-forge ipywidgetsy

import re

from happytransformer import HappyGeneration
from happytransformer import GENSettings

happy_gen = HappyGeneration("GPT-NEO", "EleutherAI/gpt-neo-2.7B")

05/29/2022 17:02:29 - INFO - happytransformer.happy_transformer -   Using model: cpu


In [4]:
def selectFromDict(options, name):

    index = 0
    indexValidList = []
    print('Select a ' + name + ':')
    for optionName in options:
        index = index + 1
        indexValidList.extend([options[optionName]])
        print(str(index) + ') ' + optionName)
    inputValid = False
    while not inputValid:
        inputRaw = input(name + ': ')
        inputNo = int(inputRaw) - 1
        if inputNo > -1 and inputNo < len(indexValidList):
            selected = indexValidList[inputNo]
            print('Selected ' +  name + ': ' + selected)
            inputValid = True
            break
        else:
            print('Please select a valid ' + name + ' number')

    return selected

In [4]:
def GenText(input_txt):
    tmp = [0.65, 0.75, 0.80, 0.85, 0.90, 0.95]
    resp = []
    for temp in tmp:
        top_k_sampling_settings = GENSettings(do_sample=True, 
                                                top_k=200, 
                                                temperature=temp,  
                                                max_length=100, 
                                                no_repeat_ngram_size=2)

        result_top_k_sampling = happy_gen.generate_text(input_txt, args=top_k_sampling_settings)

        #전처리 필요함 GenerationResult(text=' 
        result_top_k_sampling_re = result_top_k_sampling.text
        result_top_k_sampling_re.strip("'")
        result_top_k_sampling_re.lstrip("GenerationResult(text=' ")
        result_top_k_sampling_re = re.sub(r"\n", "", result_top_k_sampling_re)


        resp.append(result_top_k_sampling_re)
    return resp

In [5]:
def ExceptionInput(input_txt):
    if (input_txt.isdigit()):
        print("An incorrect input was detected. Try again.")

    elif (len(input_txt) <= 8):
        print("The sentence is too short. Try again.")

    elif (input_txt == None):
        print("Try again.")
    else:
        pass

    return input_txt

In [5]:
def Step_01():
    print("Step #1")
    print("I am using Common App OR Coalition App")
    questions = {}
    questions['CommonApp'] = 'CommonApp'
    questions['CoalitionApp'] = 'CoalitionApp'
    
    try:
        # Let user select a script
        selected_app_input = selectFromDict(questions, 'Question')
        print("selected_app_input[0] :" , selected_app_input[0])
    except:
        selected_app_input = selectFromDict(questions, 'Question')
        print("selected_app_input[0] :" , selected_app_input)
        ExceptionInput(selected_prompt_input[0])
        Step_01()
    return selected_app_input

In [6]:
Step_01()

Step #1
I am using Common App OR Coalition App
Select a Question:
1) CommonApp
2) CoalitionApp
Question: 1
Selected Question: CommonApp
selected_app_input[0] : C


'CommonApp'

In [24]:
def Step_02():
    print("Step #2")
    print("Select prompt")
    questions = {}
    questions['prompt01'] = 'Some students have a background, identity, interest, or talent that is so meaningful they believe their application would be incomplete without it. If this sounds like you, then please share your story.'
    questions['prompt02'] = 'The lessons we take from obstacles we encounter can be fundamental to later success. Recount a time when you faced a challenge, setback, or failure. How did it affect you, and what did you learn from the experience?'
    questions['prompt03'] = 'Reflect on a time when you questioned or challenged a belief or idea. What prompted your thinking? What was the outcome?'
    questions['prompt04'] = 'Reflect on something that someone has done for you that has made you happy or thankful in a surprising way. How has this gratitude affected or motivated you?'
    questions['prompt05'] = 'Discuss an accomplishment, event, or realization that sparked a period of personal growth and a new understanding of yourself or others.'
    questions['prompt06'] = 'Describe a topic, idea, or concept you find so engaging that it makes you lose all track of time. Why does it captivate you? What or who do you turn to when you want to learn more?'
    questions['prompt07'] = "Share an essay on any topic of your choice. It can be one you've already written, one that responds to a different prompt, or one of your own design."
    
    
    try:
        # Let user select a script
        selected_prompt_input = selectFromDict(questions, 'question')
    except:
        selected_prompt_input = selectFromDict(questions, 'question')
        ExceptionInput(selected_prompt_input)
        Step_02()
        
    if selected_prompt_input == """Some students have a background, identity, interest, or talent that is so meaningful they believe their application would be incomplete without it. If this sounds like you, then please share your story.""":
        background = """1)Background : What's unique about your family, education history, socioeconomic status, ethnicity, hometown, etc. You can write about your background and make sure to tell a story about how it affected your personal growth, values, beliefs, or perspectives for the future."""
        identity = """2)Identity : Your identity can be associated with your culture, race, nationality, religion, gender, disability, etc. Although your ethnic identity is determined at birth, many identities are formed by your experiences in the society and the communities you are a part of. How did your identity come about? What does it mean to you? Express why your unique identity will be a valuable addition to the college's diversity."""
        interest = """3)Interest : What is an interest that is "so meaningful" to you? Since this prompt intends to understand how unique you are, the interest you mention should be something you are passionate about or somewhat unordinary. Can you talk about an interest that defines who you are without turning it into a typical sport, music, art, or academic essay? Rather than describing your interest only, include how the lessons from this interest affected your growth as a person."""
        talent = """4)Talent : Talent could be a hard or soft skill. Making cool things, excelling in certain areas, or managing big projects are definitely solid hard skills. Also, making new friends, leading people, or empathizing with and helping others can be excellent soft skills. Avoid "bragging" or "humble bragging " when writing about your talent." Don't simply focus on how great you are, but mention the challenges you overcame or how your strength benefitted others."""
        
        topics_re = "Key Topics :" + background + identity + interest + talent
        
        # Select topic
        topics = {}
        topics['background'] = "background"
        topics['identity'] = "identity"
        topics['interest'] = "interest"
        topics['talent'] = "talent"
        
        selected_topic = selectFromDict(topics, 'topic')
    
    elif selected_prompt_input == """The lessons we take from obstacles we encounter can be fundamental to later success. Recount a time when you faced a challenge, setback, or failure. How did it affect you, and what did you learn from the experience?""":
        challenge = """1)Challenge : The prompt is not only interested in learning about the obstacle you've faced, but you should explain how you overcame such difficulty. Challenges, setbacks, and failures may seem similar in nature. However, an intriguing challenge should be a project, situation, or task that requires courage to go beyond your limitations to increase your capacity as a person."""
        setback = """2)Setback : In general, a setback is a difficulty that you encounter regardless of your will. Usually, injuries, illness, issues with mental health, parents' divorce or bankruptcy, fights, and more would be considered setbacks in life. However, instead of focusing on the difficulty, you should focus on conveying the transition in your mental state, showing how you found a way to stay resilient and optimistic, regardless of your situation."""
        failure = """3)Failure : People are afraid of failure. However, a failure commonly requires one's initiative or risk-taking initially. Because colleges want students willing to take risks to innovate things in the world, writing about your failure can be a powerful topic. Your failures may or may not lead to eventual success in your essay. However, make sure that the lesson you've learned from your failure is insightful because it shows your potential to change the world for the better."""
    
        topics_re = "Key Topics :" + challenge + setback + failure
        
        # Select topic
        topics = {}
        topics['challenge'] = "challenge"
        topics['setback'] = "setback"
        topics['failure'] = "failure"
        
        selected_topic = selectFromDict(topics, 'topic')
        
    elif selected_prompt_input == """Reflect on a time when you questioned or challenged a belief or idea. What prompted your thinking? What was the outcome?""":
        question = """1)Question : Questioning a belief or idea can involve disagreements which can be discomforting at times. Some beliefs or ideas you oppose are blatantly wrong, such as discriminating against someone. However, for the most part, people simply hold varying perspectives. So rather than showing that you are correct or more logical than others, try to demonstrate how you've been attempting to communicate your thoughts with others in a mature manner."""
        challenge = """2)Challenge : Challenging a belief or idea usually takes place when there is a need for change. Questioning can be an internal struggle, but challenging a person, system, or belief is accompanied by your action. You should explain why making such a change in your community (your family, school, town, etc.) matters to you and what tangible or intangible positive changes took place."""
    
        topics_re = "Key Topics :" + question + challenge
        
        # Select topic
        topics = {}
        topics['question'] = "question"
        topics['challenge'] = "challenge"

        selected_topic = selectFromDict(topics, 'topic')
        
    elif selected_prompt_input == """Reflect on something that someone has done for you that has made you happy or thankful in a surprising way. How has this gratitude affected or motivated you?""":
        gratitude = """1)Qratitude : Feeling grateful toward others is an essential quality of a humble and positive person. It's even better if such a feeling of gratitude leads you to empathize with others in need and fuels your action. If you are dedicated to volunteer activities and helping others, this is a perfect place to show what motivated you to give back to your community."""
        happiness = """2)Happiness : The prompt intends to learn how you don't take things for granted and know how to find happiness from the kindness of others. This is relevant to having a positive attitude; an important quality sought after by colleges. You can show your optimism towards making our society a better place and how you are contributing to your daily life."""
    
        topics_re = "Key Topics :" + gratitude + happiness
        
        # Select topic
        topics = {}
        topics['gratitude'] = "gratitude"
        topics['happiness'] = "happiness"

        selected_topic = selectFromDict(topics, 'topic')
        
        
    elif selected_prompt_input == """Discuss an accomplishment, event, or realization that sparked a period of personal growth and a new understanding of yourself or others""":
        accomplishmenht = """1)Accomplishmenht : You may think of an accomplishment as something extraordinary such as winning a big award or leading your team to victory. However, the main focus of this essay should be your personal growth, not boasting about your credentials. Instead of struggling to find a big achievement, you can start with a moment that led you to be more mature as a person, then work backward to see what caused it."""
        event = """2)Event : In many cases, personal growth is triggered by a change in your perspective. You should think of an event that humbled you and led you to become a more thoughtful and considerate person. Such an event can be a failure or an initiative you had a sense of conviction. Maybe you had a mentor, a friend, or even a stranger who taught you to see things differently. You may have felt proud or embarrassed towards the outcome of the event. Either way, your story should demonstrate a lesson and how it led you to become more mature as a person."""
        realization = """3)Realization = A realization can come from everywhere. It can be from your everyday life, reading a book, or watching a movie. Since the prompt mentions an accomplishment or event, including your own experience in the story would be more relevant than writing a review paper on a book or Youtube clip. Try to think about incidents where you've experienced hardship, challenge, or even failure and how this led to a new understanding. """
    
        topics_re = "Key Topics :" + accomplishmenht + event + realization
        
        # Select topic
        topics = {}
        topics['accomplishmenht'] = "accomplishmenht"
        topics['event'] = "event"
        topics['realization'] = "realization"
        
        selected_topic = selectFromDict(topics, 'topic')
    
    elif selected_prompt_input == """Describe a topic, idea, or concept you find so engaging that it makes you lose all track of time. Why does it captivate you? What or who do you turn to when you want to learn more""":
        topic = """1)Topic : An engaging topic varies from person to person. It can be something intellectual, social, cultural, and more. In addition, it should be a topic you are passionate about, one that you dedicate much time and effort to explore. However, please don't turn your essay into a research paper on the topic. Instead, try to explain why this topic means so much in our lives and how it has affected you in terms of your values or attitude towards life. """
        idea = """2)Idea = An engaging idea can be something academic, social, or simply creative. However, the prompt is not asking you to describe yourself as a daydreamer or a mere observer. It should be an idea meaningful enough to influence your mindset and behavior. For example, it can be something intellectual or your thoughts on how to make a better community. Furthermore, it would be more interesting to mention how your idea developed in phases or how you tried to put it into action."""
        concept = """3)Concept = A concept is perceived in your mind, and it serves as the blueprint of a more structured qualitative or physical outcome. An engaging concept becomes the backbone of your intellectual development or innovations you create in the physical world. As you explain your concept, try to share why the concept is meaningful to you as a person, scholar, or change maker for society. If you have a mentor, you can elaborate on how they have guided and influenced your growth."""
    
        topics_re = "Key Topics :" + topic + idea + concept
        
        # Select topic
        topics = {}
        topics['topic'] = "topic"
        topics['idea'] = "idea"
        topics['concept'] = "concept"
        
        selected_topic = selectFromDict(topics, 'topic')
        
    elif selected_prompt_input == """Share an essay on any topic of your choice. It can be one you've already written, one that responds to a different prompt, or one of your own design.""":
        openTopic = """1)Open topic : You can choose to write about any topic of your choice for this prompt. Because prompts 1-6 cover a wide range of topics, please make sure your topic doesn't fall under the previous prompts. While you can talk about anything, you should ensure that your essay contains the common element sought by all the prompts above and sufficiently explains your realization or lesson. The essential element in a personal statement is showing your personal growth, so you should describe how your topic led you to develop a more mature view of the world."""
       
        topics_re = "2)Key Topic :" + openTopic
        
        # Select topic
        topics = {}
        topics['openTopic'] = "openTopic"
        
        selected_topic = selectFromDict(topics, 'topic')
        
    return selected_prompt_input,topics_re, selected_topic

In [25]:
Step_02()

Step #2
Select prompt
Select a question:
1) prompt01
2) prompt02
3) prompt03
4) prompt04
5) prompt05
6) prompt06
7) prompt07
question: 7
Selected question: Share an essay on any topic of your choice. It can be one you've already written, one that responds to a different prompt, or one of your own design.
Select a topic:
1) openTopic
topic: 1
Selected topic: openTopic


("Share an essay on any topic of your choice. It can be one you've already written, one that responds to a different prompt, or one of your own design.",
 "2)Key Topic :1)Open topic : You can choose to write about any topic of your choice for this prompt. Because prompts 1-6 cover a wide range of topics, please make sure your topic doesn't fall under the previous prompts. While you can talk about anything, you should ensure that your essay contains the common element sought by all the prompts above and sufficiently explains your realization or lesson. The essential element in a personal statement is showing your personal growth, so you should describe how your topic led you to develop a more mature view of the world.",
 'openTopic')

In [10]:
# introduction
def Step_03(Selected_topic_input):
    print("")
    print("Step #3")
    print("You choese to write about ", Selected_topic_input)
    print("How does your story begin?")
    print("Where and when?")
    print("Who are in it?")
    print("What happens in the beginning?")
    print("Your response here. Minimum 50 words.")
    
    try:
        intro_input = input()
    except:
        intro_input = input()
        ExceptionInput(intro_input)
        Step_03(Selected_topic_input)
        
        
    #intro_input = input()
    print("processing...")
    
    return intro_input

<!-- nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Tue_Mar__8_18:36:24_Pacific_Standard_Time_2022
Cuda compilation tools, release 11.6, V11.6.124 Build cuda_11.6.r11.6/compiler.31057947_0 -->

# pytorch install for window
conda install pytorch torchvision torchaudio cudatoolkit=11.3 -c pytorch

# install linux
pip3 install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu113

In [11]:
#Summarization
import torch
import json 
from transformers import T5Tokenizer, T5ForConditionalGeneration, T5Config
model = T5ForConditionalGeneration.from_pretrained('t5-small')
tokenizer = T5Tokenizer.from_pretrained('t5-small')
device = torch.device('cpu')

C:\ProgramData\Anaconda3\envs\py37NLP\lib\site-packages\transformers\models\t5\tokenization_t5.py:169: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-small automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  FutureWarning,


In [12]:
def summarization(input_txt):
    preprocess_text = input_txt.strip().replace("\n","")
    t5_prepared_Text = "summarize: "+preprocess_text
    #print ("original text preprocessed: \n", preprocess_text)

    tokenized_text = tokenizer.encode(t5_prepared_Text, return_tensors="pt").to(device)


    # summmarize 
    summary_ids = model.generate(tokenized_text,
                                        num_beams=4,
                                        no_repeat_ngram_size=2,
                                        min_length=30,
                                        max_length=50, # Msx 2 senteces
                                        early_stopping=True)

    output = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return output

In [13]:
# test summarization
input_txt_test = """I add the critically measured sugary tea mixture to the gallon jar containing the slimy, white, disc-shaped layers of the symbiotic culture of bacteria and yeast.
Now to wait.  After exactly seven days, I pour the liquid into a fermentation-grade glass bottle with a ratio of 20% pomegranate juice and 80% fermented tea. I place it on my kitchen counter, periodically checking it to relieve the built-up CO2.
Finally, after an additional seventy-two hours, the time comes to try it. I crack the seal on the bottle, leaning over to smell what I assume will be a tangy, fruity, delicious pomegranate solution. and it smells like rotten eggs. The insufferable stench fills my nostrils and crushes my confidence. I'm momentarily taken aback, unable to understand how I went wrong when I followed the recipe perfectly. """

summarized_txt = summarization(input_txt_test)

print(summarized_txt)

pomegranate juice and 80% fermented tea poured into a glass bottle. after seventy-two hours, the time comes to try it, and it smells like rotten eggs.


In [14]:
# Rising Action
# input_txt : 학생이 작성한 Original Text
def Step_04(input_txt):
    print("")
    print("Step #4")
    print("What happens next? AI's ideas to get you inspired!")
    gen_sentence = GenText(input_txt)
    print("")
    print(gen_sentence)
    print("")
    print("Inspired? Let's work on the rising action. What happens after the intro? -Any conflicts, problems, or questions you've dealt with?")
    print("Your response here. Minimum 50 words.")
    
    try:
        user_response = input()
    except:
        user_response = input()
        ExceptionInput(user_response)
        Step_04(input_txt)
    
    # user_response = input()
    
    #gen_sentence : 생성된 rising aciton  6개의 문장들
    #user_response : 추가입력하는 사용자  rising action 입력문장
    return gen_sentence, user_response

In [15]:
# intro_ori_input_sent : intro 입력 원본문장
# intro_input_summarised_txt: 이미 요약된 문장으로 intro 부분
# risingAction_input_txt: Rising action 입력문장
def Step_05(intro_ori_input_sent, intro_input_summarised_txt, risingAction_input_txt):
    print("")
    print("Step #5")
    print("Your story so far.")
    risingAciton_summary_re = summarization(risingAction_input_txt)
    # Intro + Rising Action 내용 3-4문장 간추림
    print(intro_input_summarised_txt + risingAciton_summary_re)
    pre_sum_climax_sentences = intro_input_summarised_txt + risingAciton_summary_re
    
    print("")
    #Generate 되는 문장들은 학생의 original writing들 기반 (summary 아니고)
    gen_climax_sentence = GenText(intro_ori_input_sent + risingAction_input_txt)
    print(gen_climax_sentence)
    
    print("What happens next? AI's ideas to get you inspired!")
    print("")
    print("Describe the most dramatic moment in the story. What happens at the story's peak? How did your conflict, problem, or question resolve?")
    print("Your response here. Minimum 50 words.")
    
    try:
        climax_input = input()
    except:
        climax_input = input()
        ExceptionInput(climax_input)
        Step_05(intro_ori_input_sent, intro_input_summarised_txt, risingAction_input_txt)
    
    # climax_input = input()
    print("Your Climax input:", climax_input)
    
    summarised_climax = summarization(climax_input)
    
    # risingAciton_summary_re : summary rising action
    # gen_climax_sentence : 생성된 Climax idea sentences
    # climax_input : Use input climax
    # summarised_climax : summary climax
    return risingAciton_summary_re, gen_climax_sentence, climax_input, summarised_climax
    

In [16]:
def Step_06(sum_intro, sum_rasingAciton, sum_climax, intro_ori_input_sent, risingAction_input_txt, climax_input_txt):
    print("")
    print("Step #6")
    print("How did your conflict, problem, or question resolve?")
    # show summary of Intro + Rasing Action + Climax
    step_6_que_re = sum_intro + sum_rasingAciton + sum_climax
    #print(step_6_que_re)
    print("")
    print("What happens next? AI's ideas to get you inspired!")
    gen_resolution = GenText(intro_ori_input_sent + risingAction_input_txt + climax_input_txt)
    print("Generated resolution sentences :", gen_resolution)
    print("")
    print("""Describe how your story ends. What happens at the end of your story? -How did this experience change your perspective?""")
    print("Your response here. Minimum 50 words.")
    
    try:
        Resolution_input = input()
    except:
        Resolution_input = input()
        ExceptionInput(Resolution_input)
        Step_06(sum_intro, sum_rasingAciton, sum_climax, intro_ori_input_sent, risingAction_input_txt, climax_input_txt)
    
    # Resolution_input = input()
    print("Your resolution input :", Resolution_input)
    
    # gen_resolution : 생성한 Resolution sentences
    # Resolution_input : User input resolution
    return gen_resolution, Resolution_input

doc = """
         Supervised learning is the machine learning task of 
         learning a function that maps an input to an output based 
         on example input-output pairs.[1] It infers a function 
         from labeled training data consisting of a set of 
         training examples.[2] In supervised learning, each 
         example is a pair consisting of an input object 
         (typically a vector) and a desired output value (also 
         called the supervisory signal). A supervised learning 
         algorithm analyzes the training data and produces an 
         inferred function, which can be used for mapping new 
         examples. An optimal scenario will allow for the algorithm 
         to correctly determine the class labels for unseen 
         instances. This requires the learning algorithm to  
         generalize from the training data to unseen situations 
         in a 'reasonable' way (see inductive bias).
      """

from sklearn.feature_extraction.text import CountVectorizer

n_gram_range = (1, 1)
stop_words = "english"

# Extract candidate words/phrases
count = CountVectorizer(ngram_range=n_gram_range, stop_words=stop_words).fit([doc])
candidates = count.get_feature_names()

# https://www.sbert.net/docs/pretrained_models.html#average-word-embeddings-models

from sentence_transformers import SentenceTransformer

model = SentenceTransformer('xlm-r-distilroberta-base-paraphase-v1')
doc_embedding = model.encode([doc])
candidate_embeddings = model.encode(candidates)


from sklearn.metrics.pairwise import cosine_similarity

top_n = 5
distances = cosine_similarity(doc_embedding, candidate_embeddings)
keywords = [candidates[index] for index in distances.argsort()[0][-top_n:]]

def KeyTopic():
    

In [17]:
def Run():
    Step_01()
    
    print("")
    
    selected_topic = Step_02()
    
    print("")
    
    #intro_input_txt:  학생이 작성한 intro sentence
    intro_input_txt = Step_03(selected_topic)
    
    print("")
    
    #intro summary sentence
    intro_summarized_txt = summarization(intro_input_txt)
    
    print("")
    
    print('summary intro :', intro_summarized_txt)
    
    print("")

    result = Step_04(intro_input_txt)
    #gen_sentence  > result[0]: 생성된 6개의 risng action 문장들
    #user_response  > result[1]: 사용자 rising action입력문장
    
    print("")
    
    step_5_re = Step_05(intro_input_txt, intro_summarized_txt, result[1])
    print("Generated 6 sentences of the climax idea", step_5_re[1]) # 생성된 Climax idea sentences
    print("")
    print("User climax input sentence :", step_5_re[2]) # User climax input
    print("")
    
    sum_intro = intro_summarized_txt
    sum_rasingAciton = step_5_re[0]
    sum_climax = step_5_re[2] 
    intro_ori_input_sent = intro_input_txt
    risingAction_input_txt = result[1]
    climax_input_txt = step_5_re[2]
    
    gen_resolution_re = Step_06(sum_intro, sum_rasingAciton, sum_climax, intro_ori_input_sent, risingAction_input_txt, climax_input_txt)
    print("")
    # gen_resolution_re[0] : 생성한 Resolution sentences
    # gen_resolution_re[1] : User input resolution
    print("Generated the resolution sentences :", gen_resolution_re[0])
    print("")
    print("User input resolution sentence :", gen_resolution_re[1])
    print("")
    
    return gen_resolution_re[0], gen_resolution_re[1]

In [18]:
Run()


Step #1
I am using Common App OR Coalition App
Select a Question:
1) CommonApp
2) CoalitionApp
Question: 1
Selected Question: CommonApp

Step #2
Select prompt
Select a Question:
1) prompt01
2) prompt02
3) prompt03
4) prompt04
5) prompt05
6) prompt06
7) prompt07
Question: 2
Selected Question: The lessons we take from obstacles we encounter can be fundamental to later success. Recount a time when you faced a challenge, setback, or failure. How did it affect you, and what did you learn from the experience?


Step #3
You choese to write about  The lessons we take from obstacles we encounter can be fundamental to later success. Recount a time when you faced a challenge, setback, or failure. How did it affect you, and what did you learn from the experience?
How does your story begin?
Where and when?
Who are in it?
What happens in the beginning?
Your response here. Minimum 50 words.
You choese to write about  The lessons we take from obstacles we encounter can be fundamental to later success.

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



summary intro : you choese to write about the lessons we take from obstacles we encounter. how did it affect you, and what did you learn from the experience?


Step #4
What happens next? AI's ideas to get you inspired!


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



['In this book, I share my personal experiences and lessons learned from setbacks, difficulties, challenges, failures, losses, trials, tribulations, disappointments, bumps along the road, obstacles, roadblocks, distractions, trouble, pains, heartbreaks, setbacks and failures. Some of my episodes of failed dreams, failed plans, lost opportunities, mistakes and losses have been very painful, but I have also been able to learn some very important lessons about what it means to be successful in life and', 'It is important to note that this writing assignment is in no way a test of your knowledge of the subject. In fact, you may not know the answer to this question. You may have no idea. Or you know one answer and don’t have another. The point is that it is a chance for you to begin to build a narrative, to draw out your thoughts and feelings from your experiences.When you are writing, it helps to use your own experiences and emotions to', "_________________If you are a victim of domestic 

...

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated 6 sentences of the climax idea [', I\'m sorry. But I hope that you can see that no matter what the situation, you need to let go of your anger, hurt, fear, pain, grief, guilt, sadness, disappointment, hopelessness, anxiety, etc. We all have them. You don\'t have to be a monster, but it\'s okay to say "I\'m angry!" and "That hurts!" When you do, your heart will be open and receptive to hear the message your body is sending you...', ', I am beyond disgusted by your behavior. I hope that a day will come when I will be able to speak my truth without fear of retaliation, but I doubt it. This is a work in progress. ____________________"_____________________"______________________________________________" _______________________________________________________.___________________ _________________________"___________________________________________________________________________________ ______________________________________ ____ ______ _______"_______________"___"__"_______ "___""

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated resolution sentences : [" to be committed.• what you would like to learn.'• how you learned to get to the conclusion that you came to. _____________.If the victim is a perpetrator, how does the perpetrator learn the behaviors that have caused the harm?What are the lessons that the offender learns? What strategies does he or she use to continue the cycle of violence?How does this cycle affect the victims, who are frequently affected by their behavior?• What is the relationship between the two ", ' scene, victim or perpetrator.• what we are interested in from our own stories and experiences".In the first section you describe the situations that have caused you trouble. By writing about them, these experiences can become a vital part of your personal narrative and have the potential to become your life stories. It doesn’t matter if you don‘t want them to be your main story. Write about any situations or situation that you have been through in life. You have to start from', ", wi

([" to be committed.• what you would like to learn.'• how you learned to get to the conclusion that you came to. _____________.If the victim is a perpetrator, how does the perpetrator learn the behaviors that have caused the harm?What are the lessons that the offender learns? What strategies does he or she use to continue the cycle of violence?How does this cycle affect the victims, who are frequently affected by their behavior?• What is the relationship between the two ",
  ' scene, victim or perpetrator.• what we are interested in from our own stories and experiences".In the first section you describe the situations that have caused you trouble. By writing about them, these experiences can become a vital part of your personal narrative and have the potential to become your life stories. It doesn’t matter if you don‘t want them to be your main story. Write about any situations or situation that you have been through in life. You have to start from',
  ", with what concerns, if any.•th